In [94]:
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import sweetviz as sv
from sklearn import datasets
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML

data = datasets.load_iris()

original_df = pd.read_csv("./data/data.csv")
blind_df = pd.DataFrame({
    'A': original_df['sepal length (cm)'].values,
    'B': original_df['sepal width (cm)'].values,
    'C': original_df['petal length (cm)'].values,
    'D': original_df['petal width (cm)'].values,
    'target': original_df['class'].values
})
blind_df.set_index('target')
blind_df['target'] = blind_df['target'].apply(lambda x: 0 if 'Iris-setosa' in x else 1 if 'Iris-versicolor' in x else 2 if 'Iris-virginica' in x else x)
# detect duplicates except first one of each
duplicated = blind_df[blind_df.duplicated()]
# remove duplicates
blind_df = blind_df.drop_duplicates()

# COlumn B has low correlation with target and other columns, we can drop it from the dataset
blind_df = blind_df.drop('B', axis=True)

# profile = ProfileReport(blind_df, title='Iris Profiling Report', explorative=True)
# profile.to_widgets() # Internal report
# # profile.to_notebook_iframe() # display html
# profile.to_file('./analysis/pandas-profiling.html') # save the report to html

# Send the data to autoML
X = pd.DataFrame(blind_df.drop('target',axis=True).values, columns=['A','C','D'])
y = pd.Series(blind_df['target'].values, name='target')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

automl = AutoML(algorithms=['Decision Tree', 'Linear', 'Random Forest'], total_time_limit=5*60)
automl.fit(X_train, y_train)

y_predicted = automl.predict(X_test)
print(pd.DataFrame({'Predicted': y_predicted['label'], 'Target': np.array(y_test)}))



Create directory AutoML_2
AutoML task to be solved: multiclass_classification
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest']
AutoML will optimize for metric: logloss
1_DecisionTree final logloss 0.48212734438216076 time 4.87 seconds
2_Linear final logloss 0.1570937244501868 time 3.24 seconds
3_Default_RandomForest final logloss 0.1787654027643232 time 3.19 seconds
Ensemble final logloss 0.1267120944985365 time 0.05 seconds
   Predicted  Target
0          2       2
1          2       2
2          1       1
3          2       2
4          2       2
5          0       0
6          0       0
7          0       0
8          0       0
9          2       2
10         1       1
11         1       1
12         1       1
13         2       2
14         1       1


AttributeError: 'list' object has no attribute 'dtypes'